In [1]:
import numpy as np
import pandas as pd

In [9]:
df = pd.read_csv('Loan_default.csv')
df = df[['LoanAmount', 'Income', 'InterestRate', 'DTIRatio', 'Age', 'LoanPurpose', 'HasMortgage', 'HasDependents', 'CreditScore']]

In [11]:
df['HasMortgage'] = df['HasMortgage'].replace(['Yes', 'No'], [1, 0])
df['HasDependents'] = df['HasDependents'].replace(['Yes', 'No'], [1, 0])
df['LoanPurpose'] = df['LoanPurpose'].replace(['Business', 'Home', 'Education', 'Other', 'Auto'], [0, 1, 2, 3, 4])

C:\Users\dipan\AppData\Local\Temp\ipykernel_3776\1617550533.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['HasMortgage'] = df['HasMortgage'].replace(['Yes', 'No'], [1, 0])
C:\Users\dipan\AppData\Local\Temp\ipykernel_3776\1617550533.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['HasDependents'] = df['HasDependents'].replace(['Yes', 'No'], [1, 0])
C:\Users\dipan\AppData\Local\Temp\ipykernel_3776\1617550533.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. 

In [13]:
from sklearn.preprocessing import Normalizer
norm = Normalizer()

In [19]:
df[['Income', 'LoanAmount', 'CreditScore', 'InterestRate', 'DTIRatio']] = norm.fit_transform(df[['Income', 'LoanAmount', 'CreditScore', 'InterestRate', 'DTIRatio']])

In [33]:
df.head()

,Age,LoanPurpose,HasMortgage,HasDependents,CreditScore,DTIRatio*InterestRate,LoanAmount/Income
0,56,3,1,1,0.005212,-21.118982,0.826849
1,69,3,0,0,0.003411,-22.430208,0.969918
2,46,4,1,1,0.002925,-22.010739,0.775223
3,32,0,0,0,0.013535,-21.340090,0.726974
4,60,4,0,1,0.028264,-18.474671,0.936927


In [27]:
df['DTIRatio*InterestRate'] = (np.log(df['DTIRatio'] * df['InterestRate']))
df['LoanAmount/Income'] = (np.abs(np.log(df['LoanAmount']/df['Income']))**0.30)

In [31]:
df.drop(columns = ['LoanAmount', 'Income', 'InterestRate', 'DTIRatio'], inplace=True)

In [35]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [37]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns = ['CreditScore']),
                                                    df['CreditScore'],
                                                    test_size=0.2,
                                                    random_state=42)

In [41]:
# Convert data into LightGBM Dataset format
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# Define LightGBM-XT Parameters
params = {
    "boosting_type": "goss",  # Gradient-Based One-Side Sampling (for LightGBM-XT)
    "objective": "regression",  # Use "binary" for classification
    "metric": "r2",  # R-squared metric
    "learning_rate": 0.05,
    "num_leaves": 64,
    "max_depth": -1,
    "feature_fraction": 0.8,
    "bagging_fraction": 0.8,
    "lambda_l1": 0.1,
    "lambda_l2": 0.1,
    "verbose": -1
}

# Train the model
model = lgb.train(params, train_data, valid_sets=[test_data])

In [43]:
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)

print(f"R-squared Score: {r2:.4f}")


R-squared Score: 0.5428
